# F1 Aerodynamics Parameter Sweep Study

This notebook performs systematic parameter sweeps over:
- **Ride height**: Ground clearance of the car body
- **Reynolds number**: Flow regime characterization

Results are automatically saved for each configuration.

In [6]:
import matplotlib.pyplot as plt
import numpy as np
from lbm_core import LBMSolver
from boundaries import TunnelBoundaries
from analysis import plot_complexity_dashboard
from aerodynamics import calculate_lift_drag, check_ground_effect
import json
import pickle
from datetime import datetime
import os
from pathlib import Path
import pandas as pd

## Configuration and Parameter Sweep Setup

In [7]:
# --- BASE CONFIGURATION ---
NX, NY = 500, 200
GROUND_TYPE = "no_slip"
SIMULATION_STEPS = 3000
PRINT_INTERVAL = 100

# --- PARAMETER SWEEP RANGES ---
# Ride heights to test (ground clearance in grid units)
RIDE_HEIGHTS = [15, 19, 23, 27, 31]  # Lower = closer to ground

# Reynolds numbers to test
REYNOLDS_NUMBERS = [10000]

# --- OUTPUT CONFIGURATION ---
# Create results directory with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
results_dir = Path(f"results_{timestamp}")
results_dir.mkdir(exist_ok=True)

# Subdirectories for different output types
(results_dir / "flow_fields").mkdir(exist_ok=True)
(results_dir / "visualizations").mkdir(exist_ok=True)
(results_dir / "raw_data").mkdir(exist_ok=True)

print(f"Results will be saved to: {results_dir}")
print(f"Total simulations to run: {len(RIDE_HEIGHTS) * len(REYNOLDS_NUMBERS)}")
print(f"Ride heights: {RIDE_HEIGHTS}")
print(f"Reynolds numbers: {REYNOLDS_NUMBERS}")

Results will be saved to: results_20260126_165429
Total simulations to run: 5
Ride heights: [15, 19, 23, 27, 31]
Reynolds numbers: [10000]


## Helper Functions for Results Management

In [8]:
class SimulationResults:
    """Container for simulation results with save/load functionality"""
    
    def __init__(self, ride_height, reynolds, nx, ny):
        self.ride_height = ride_height
        self.reynolds = reynolds
        self.nx = nx
        self.ny = ny
        self.timestamp = datetime.now().isoformat()
        
        # Time series data
        self.lift_history = []
        self.drag_history = []
        self.step_history = []
        
        # Final state
        self.final_velocity_field = None
        self.final_lift = None
        self.final_drag = None
        
        # Statistics
        self.mean_lift = None
        self.mean_drag = None
        self.std_lift = None
        self.std_drag = None
        self.lift_to_drag_ratio = None
        
    def add_step(self, step, lift, drag):
        """Add data point from simulation step"""
        self.step_history.append(step)
        self.lift_history.append(lift)
        self.drag_history.append(drag)
    
    def finalize(self, velocity_field):
        """Calculate final statistics and store final state"""
        self.final_velocity_field = velocity_field.copy()
        self.final_lift = self.lift_history[-1] if self.lift_history else None
        self.final_drag = self.drag_history[-1] if self.drag_history else None
        
        # Calculate statistics from last 1000 steps (steady state)
        if len(self.lift_history) > 1000:
            steady_lift = self.lift_history[-1000:]
            steady_drag = self.drag_history[-1000:]
        else:
            steady_lift = self.lift_history
            steady_drag = self.drag_history
        
        self.mean_lift = np.mean(steady_lift)
        self.mean_drag = np.mean(steady_drag)
        self.std_lift = np.std(steady_lift)
        self.std_drag = np.std(steady_drag)
        
        if self.mean_drag != 0:
            self.lift_to_drag_ratio = abs(self.mean_lift / self.mean_drag)
        else:
            self.lift_to_drag_ratio = None
    
    def save(self, base_dir):
        """Save results to disk"""
        base_dir = Path(base_dir)
        prefix = f"rh{self.ride_height}_re{self.reynolds}"
        
        # Save metadata and statistics as JSON
        metadata = {
            'ride_height': self.ride_height,
            'reynolds': self.reynolds,
            'nx': self.nx,
            'ny': self.ny,
            'timestamp': self.timestamp,
            'final_lift': float(self.final_lift) if self.final_lift is not None else None,
            'final_drag': float(self.final_drag) if self.final_drag is not None else None,
            'mean_lift': float(self.mean_lift) if self.mean_lift is not None else None,
            'mean_drag': float(self.mean_drag) if self.mean_drag is not None else None,
            'std_lift': float(self.std_lift) if self.std_lift is not None else None,
            'std_drag': float(self.std_drag) if self.std_drag is not None else None,
            'lift_to_drag_ratio': float(self.lift_to_drag_ratio) if self.lift_to_drag_ratio is not None else None,
        }
        
        with open(base_dir / "raw_data" / f"{prefix}_metadata.json", 'w') as f:
            json.dump(metadata, f, indent=2)
        
        # Save time series data
        time_series = {
            'steps': self.step_history,
            'lift': self.lift_history,
            'drag': self.drag_history
        }
        np.savez_compressed(
            base_dir / "raw_data" / f"{prefix}_timeseries.npz",
            **time_series
        )
        
        # Save velocity field
        if self.final_velocity_field is not None:
            np.save(
                base_dir / "flow_fields" / f"{prefix}_velocity.npy",
                self.final_velocity_field
            )
        
        print(f"  Saved: {prefix}")
        return prefix


def create_geometry(bounds, ride_height):
    """Create F1 car geometry with specified ride height"""
    bounds.add_ground(type=GROUND_TYPE)
    
    # Front wing - height adjusts with ride_height
    bounds.add_f1_wing_proxy(
        x_pos=150,
        height=ride_height,
        length=60,
        slope=0.45
    )
    
    # Main body - height also scales with ride_height
    bounds.add_rectangular_obstacle(
        x_start=210,
        y_start=ride_height + 1,
        length=120,
        height=20
    )
    
    # Underbody diffuser
    bounds.add_reverse_triangle(
        x_pos=240,
        height=ride_height + 20,
        length=90,
        slope=2/9
    )
    
    # Rear wing
    bounds.add_f1_wing_proxy(
        x_pos=300,
        height=ride_height + 30,
        length=30,
        slope=0.2
    )


def plot_results_comparison(results_list, save_dir):
    """Create comparison plots for all results"""
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Extract data for plotting
    ride_heights = sorted(list(set(r.ride_height for r in results_list)))
    reynolds_nums = sorted(list(set(r.reynolds for r in results_list)))
    
    # Plot 1: Mean Lift vs Ride Height (for each Re)
    ax = axes[0, 0]
    for re in reynolds_nums:
        data = [(r.ride_height, r.mean_lift) for r in results_list if r.reynolds == re]
        data.sort()
        rh, lift = zip(*data)
        ax.plot(rh, lift, 'o-', label=f'Re={re}', linewidth=2, markersize=8)
    ax.set_xlabel('Ride Height (grid units)', fontsize=12)
    ax.set_ylabel('Mean Downforce (negative = downforce)', fontsize=12)
    ax.set_title('Downforce vs Ride Height', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='k', linestyle='--', alpha=0.5)
    
    # Plot 2: Mean Drag vs Ride Height (for each Re)
    ax = axes[0, 1]
    for re in reynolds_nums:
        data = [(r.ride_height, r.mean_drag) for r in results_list if r.reynolds == re]
        data.sort()
        rh, drag = zip(*data)
        ax.plot(rh, drag, 's-', label=f'Re={re}', linewidth=2, markersize=8)
    ax.set_xlabel('Ride Height (grid units)', fontsize=12)
    ax.set_ylabel('Mean Drag', fontsize=12)
    ax.set_title('Drag vs Ride Height', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 3: L/D Ratio vs Ride Height
    ax = axes[1, 0]
    for re in reynolds_nums:
        data = [(r.ride_height, r.lift_to_drag_ratio) for r in results_list 
                if r.reynolds == re and r.lift_to_drag_ratio is not None]
        if data:
            data.sort()
            rh, ld = zip(*data)
            ax.plot(rh, ld, '^-', label=f'Re={re}', linewidth=2, markersize=8)
    ax.set_xlabel('Ride Height (grid units)', fontsize=12)
    ax.set_ylabel('|Lift/Drag| Ratio', fontsize=12)
    ax.set_title('Aerodynamic Efficiency (Higher is Better)', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 4: Heatmap of Downforce
    ax = axes[1, 1]
    lift_matrix = np.zeros((len(reynolds_nums), len(ride_heights)))
    for i, re in enumerate(reynolds_nums):
        for j, rh in enumerate(ride_heights):
            result = next((r for r in results_list if r.reynolds == re and r.ride_height == rh), None)
            if result:
                lift_matrix[i, j] = result.mean_lift
    
    im = ax.imshow(lift_matrix, aspect='auto', cmap='RdBu_r', origin='lower')
    ax.set_xticks(range(len(ride_heights)))
    ax.set_yticks(range(len(reynolds_nums)))
    ax.set_xticklabels(ride_heights)
    ax.set_yticklabels(reynolds_nums)
    ax.set_xlabel('Ride Height (grid units)', fontsize=12)
    ax.set_ylabel('Reynolds Number', fontsize=12)
    ax.set_title('Downforce Heatmap (Blue = Strong Downforce)', fontsize=14, fontweight='bold')
    plt.colorbar(im, ax=ax, label='Mean Downforce')
    
    # Add text annotations
    for i in range(len(reynolds_nums)):
        for j in range(len(ride_heights)):
            text = ax.text(j, i, f'{lift_matrix[i, j]:.3f}',
                          ha="center", va="center", color="black", fontsize=8)
    
    plt.tight_layout()
    plt.savefig(save_dir / "visualizations" / "parameter_sweep_summary.png", dpi=300, bbox_inches='tight')
    plt.close()
    print("\nSaved summary comparison plot")

## Run Parameter Sweep

This cell performs the full parameter sweep. Each simulation:
1. Creates the geometry with the specified ride height
2. Runs the LBM simulation for the specified Reynolds number
3. Tracks lift and drag forces throughout
4. Saves all results automatically

In [9]:
# Storage for all results
all_results = []

total_sims = len(RIDE_HEIGHTS) * len(REYNOLDS_NUMBERS)
sim_counter = 0

print("="*70)
print("STARTING PARAMETER SWEEP")
print("="*70)

for reynolds in REYNOLDS_NUMBERS:
    for ride_height in RIDE_HEIGHTS:
        sim_counter += 1
        print(f"\n[{sim_counter}/{total_sims}] Re={reynolds}, Ride Height={ride_height}")
        print("-" * 70)
        
        # Initialize results container
        results = SimulationResults(ride_height, reynolds, NX, NY)
        
        # Setup solver and boundaries
        solver = LBMSolver(NX, NY, reynolds, u_inlet=0.1)
        bounds = TunnelBoundaries(NX, NY)
        create_geometry(bounds, ride_height)
        
        # Define analysis region (adjust if geometry changes significantly)
        relevant_x_start = 145
        relevant_x_end = 335
        relevant_y_start = 5
        relevant_y_end = min(75, ride_height + 60)
        
        # Run simulation
        print(f"  Running {SIMULATION_STEPS} steps...")
        for step in range(SIMULATION_STEPS):
            solver.collide_and_stream(bounds.mask)
            bounds.apply_inlet_outlet(solver)
            
            # Calculate forces every step
            if step % PRINT_INTERVAL == 0 or step == SIMULATION_STEPS - 1:
                fx, fy = calculate_lift_drag(
                    solver, bounds,
                    x_start=relevant_x_start,
                    x_end=relevant_x_end,
                    y_start=relevant_y_start,
                    y_end=relevant_y_end
                )
                results.add_step(step, fy, fx)
                
                if step % (PRINT_INTERVAL * 5) == 0:
                    print(f"    Step {step}/{SIMULATION_STEPS} - Lift: {fy:.4f}, Drag: {fx:.4f}")
        
        # Finalize and save results
        velocity_mag = np.sqrt(solver.u[:,:,0]**2 + solver.u[:,:,1]**2)
        results.finalize(velocity_mag)
        results.save(results_dir)
        all_results.append(results)
        
        print(f"  ✓ Complete: Mean Lift={results.mean_lift:.4f}, Mean Drag={results.mean_drag:.4f}")
        if results.lift_to_drag_ratio:
            print(f"  ✓ L/D Ratio: {results.lift_to_drag_ratio:.4f}")

print("\n" + "="*70)
print("PARAMETER SWEEP COMPLETE")
print("="*70)

STARTING PARAMETER SWEEP

[1/5] Re=10000, Ride Height=15
----------------------------------------------------------------------
  Running 3000 steps...
    Step 0/3000 - Lift: 0.3333, Drag: -21.6000
    Step 500/3000 - Lift: -0.6367, Drag: -6.5615
    Step 1000/3000 - Lift: 0.4847, Drag: -6.2285
    Step 1500/3000 - Lift: -0.9734, Drag: -4.3719
    Step 2000/3000 - Lift: -0.4380, Drag: -4.4336
    Step 2500/3000 - Lift: -0.7357, Drag: -3.0986
  Saved: rh15_re10000
  ✓ Complete: Mean Lift=-0.4121, Mean Drag=-5.8294
  ✓ L/D Ratio: 0.0707

[2/5] Re=10000, Ride Height=19
----------------------------------------------------------------------
  Running 3000 steps...
    Step 0/3000 - Lift: 0.3333, Drag: -21.6000
    Step 500/3000 - Lift: -0.5133, Drag: -6.4420
    Step 1000/3000 - Lift: 0.6095, Drag: -6.0152
    Step 1500/3000 - Lift: -0.9637, Drag: -3.8785
    Step 2000/3000 - Lift: -0.1428, Drag: -4.5416
    Step 2500/3000 - Lift: -0.8713, Drag: -2.9476
  Saved: rh19_re10000
  ✓ Complete: 

## Generate Summary Plots and Export Data

In [10]:
# Create comparison plots
plot_results_comparison(all_results, results_dir)

# Export summary table as CSV
summary_data = []
for r in all_results:
    summary_data.append({
        'ride_height': r.ride_height,
        'reynolds': r.reynolds,
        'mean_lift': r.mean_lift,
        'mean_drag': r.mean_drag,
        'std_lift': r.std_lift,
        'std_drag': r.std_drag,
        'lift_to_drag_ratio': r.lift_to_drag_ratio,
    })

df = pd.DataFrame(summary_data)
df = df.sort_values(['reynolds', 'ride_height'])
df.to_csv(results_dir / "summary_table.csv", index=False)

print("\n" + "="*70)
print("SUMMARY TABLE")
print("="*70)
print(df.to_string(index=False))
print(f"\nFull results saved to: {results_dir}")
print(f"CSV exported to: {results_dir / 'summary_table.csv'}")


Saved summary comparison plot

SUMMARY TABLE
 ride_height  reynolds  mean_lift  mean_drag  std_lift  std_drag  lift_to_drag_ratio
          15     10000  -0.412145  -5.829433  0.730264  3.659237            0.070701
          19     10000  -0.268902  -5.709246  0.634016  3.657207            0.047099
          23     10000  -0.282660  -5.628537  0.666205  3.645131            0.050219
          27     10000  -0.280429  -5.514797  0.708659  3.658909            0.050850
          31     10000  -0.225062  -5.401263  0.763513  3.674758            0.041668

Full results saved to: results_20260126_165429
CSV exported to: results_20260126_165429\summary_table.csv


## Visualize Individual Cases

Generate flow field visualizations for each configuration

In [11]:
print("Generating individual flow field visualizations...\n")

for result in all_results:
    fig, axes = plt.subplots(2, 1, figsize=(16, 10))
    
    # Flow field
    ax = axes[0]
    im = ax.imshow(result.final_velocity_field, origin='lower', cmap='magma', vmin=0, vmax=0.15)
    ax.set_title(f'Velocity Field - Re={result.reynolds}, Ride Height={result.ride_height}', 
                 fontsize=14, fontweight='bold')
    ax.set_xlabel('X (grid units)')
    ax.set_ylabel('Y (grid units)')
    plt.colorbar(im, ax=ax, label='Velocity Magnitude')
    
    # Force history
    ax = axes[1]
    ax.plot(result.step_history, result.lift_history, 'b-', linewidth=1.5, label='Lift (Downforce)')
    ax.plot(result.step_history, result.drag_history, 'r-', linewidth=1.5, label='Drag')
    ax.axhline(y=result.mean_lift, color='b', linestyle='--', alpha=0.5, label=f'Mean Lift: {result.mean_lift:.4f}')
    ax.axhline(y=result.mean_drag, color='r', linestyle='--', alpha=0.5, label=f'Mean Drag: {result.mean_drag:.4f}')
    ax.set_xlabel('Simulation Step', fontsize=12)
    ax.set_ylabel('Force', fontsize=12)
    ax.set_title('Aerodynamic Forces vs Time', fontsize=14, fontweight='bold')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    filename = f"rh{result.ride_height}_re{result.reynolds}_visualization.png"
    plt.savefig(results_dir / "visualizations" / filename, dpi=200, bbox_inches='tight')
    plt.close()
    print(f"  Saved: {filename}")

print("\nAll visualizations complete!")

Generating individual flow field visualizations...

  Saved: rh15_re10000_visualization.png
  Saved: rh19_re10000_visualization.png
  Saved: rh23_re10000_visualization.png
  Saved: rh27_re10000_visualization.png
  Saved: rh31_re10000_visualization.png

All visualizations complete!


## Find Optimal Configuration

In [12]:
# Find configuration with maximum downforce (most negative lift)
max_downforce = min(all_results, key=lambda r: r.mean_lift)

# Find configuration with best L/D ratio
valid_ld = [r for r in all_results if r.lift_to_drag_ratio is not None]
if valid_ld:
    best_efficiency = max(valid_ld, key=lambda r: r.lift_to_drag_ratio)
else:
    best_efficiency = None

print("="*70)
print("OPTIMAL CONFIGURATIONS")
print("="*70)
print(f"\nMaximum Downforce:")
print(f"  Ride Height: {max_downforce.ride_height}")
print(f"  Reynolds: {max_downforce.reynolds}")
print(f"  Mean Lift: {max_downforce.mean_lift:.4f}")
print(f"  Mean Drag: {max_downforce.mean_drag:.4f}")

if best_efficiency:
    print(f"\nBest Aerodynamic Efficiency (L/D):")
    print(f"  Ride Height: {best_efficiency.ride_height}")
    print(f"  Reynolds: {best_efficiency.reynolds}")
    print(f"  L/D Ratio: {best_efficiency.lift_to_drag_ratio:.4f}")
    print(f"  Mean Lift: {best_efficiency.mean_lift:.4f}")
    print(f"  Mean Drag: {best_efficiency.mean_drag:.4f}")

# Save optimal configs to file
optimal_configs = {
    'max_downforce': {
        'ride_height': max_downforce.ride_height,
        'reynolds': max_downforce.reynolds,
        'mean_lift': float(max_downforce.mean_lift),
        'mean_drag': float(max_downforce.mean_drag)
    }
}

if best_efficiency:
    optimal_configs['best_efficiency'] = {
        'ride_height': best_efficiency.ride_height,
        'reynolds': best_efficiency.reynolds,
        'lift_to_drag_ratio': float(best_efficiency.lift_to_drag_ratio),
        'mean_lift': float(best_efficiency.mean_lift),
        'mean_drag': float(best_efficiency.mean_drag)
    }

with open(results_dir / "optimal_configurations.json", 'w') as f:
    json.dump(optimal_configs, f, indent=2)

print(f"\nOptimal configurations saved to: {results_dir / 'optimal_configurations.json'}")

OPTIMAL CONFIGURATIONS

Maximum Downforce:
  Ride Height: 15
  Reynolds: 10000
  Mean Lift: -0.4121
  Mean Drag: -5.8294

Best Aerodynamic Efficiency (L/D):
  Ride Height: 15
  Reynolds: 10000
  L/D Ratio: 0.0707
  Mean Lift: -0.4121
  Mean Drag: -5.8294

Optimal configurations saved to: results_20260126_165429\optimal_configurations.json
